<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AAI_PA_LLM_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import time
import json
from sklearn.linear_model import LinearRegression
import numpy as np

# Provided setup
import google.generativeai as genai
from google.colab import userdata # Keep this as per your instruction

# --- API Key Setup (as provided by you, directly used) ---
GOOGLE_API_KEY = userdata.get('GEMINI')
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Generative AI configured successfully using Colab Secrets.")
else:
    print("WARNING: GOOGLE_API_KEY not found in Colab Secrets. Please ensure 'GEMINI' secret is set.")
    print("API calls will likely fail. Proceeding with unconfigured API.")

# --- Agent Configuration ---
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash" # As specified by you

# Initialize the Gemini model for general responses and agentic decisions
AGENTIC_MODEL = None
RESPONDER_MODEL = None
try:
    if GOOGLE_API_KEY: # Only attempt if API key is present
        AGENTIC_MODEL = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
        RESPONDER_MODEL = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
        print(f"Gemini model '{AgentConfig.LLM_MODEL_NAME}' initialized for agentic and response generation.")
    else:
        print("Skipping Gemini model initialization due to missing API key.")
except Exception as e:
    print(f"ERROR: Failed to initialize Gemini model. Please check your API key and model name. Error: {e}")
    print("Proceeding with dummy models for LLM interactions.")

# --- Real LLM Interaction Function (Now uses actual API calls, without temperature) ---
def call_gemini_llm(model, prompt: str) -> str: # Removed temperature parameter
    """
    Makes a call to the Gemini LLM and returns its text response.
    Handles potential API errors.
    """
    if model is None:
        print("[LLM ERROR] Gemini model not initialized. Using dummy response.")
        # Fallback dummy responses for LLM reasoning
        if "initial planning strategy" in prompt.lower():
            return "STRATEGY: Balanced Planning" # Dummy response now matches expected format
        elif "evaluate route impact" in prompt.lower():
            return "RECOMMENDATION: direct_route (dummy reason)" # Dummy response now matches expected format
        elif "generate final flight plan" in prompt.lower():
            return "Dummy Flight Plan Summary (Generated by Dummy LLM)"
        return "Dummy LLM Response."

    print(f"\n[LLM] Calling Gemini model with prompt (first 100 chars): \"{prompt[:100]}...\"")
    try:
        # Removed temperature=temperature from the call
        response = model.generate_content(prompt)
        if response and response.text:
            return response.text
        else:
            print(f"[LLM ERROR] Gemini returned no text. Response: {response}")
            return "LLM THOUGHT: No text in response. ACTION: Cannot proceed."
    except Exception as e:
        print(f"[LLM API ERROR] An error occurred during Gemini API call: {e}")
        return "LLM THOUGHT: API call failed. ACTION: Fallback to a default or simpler plan."


# --- 2. Simulate Predictive Analytics (Tools for the Agent) ---

class PredictiveModels:
    def __init__(self):
        # A very simple linear regression model for fuel consumption
        self.fuel_model = LinearRegression()
        # Train a dummy model
        X_fuel = np.array([
            [100, 10, 30], [100, -10, 35],
            [500, 20, 30], [500, -20, 40],
            [1000, 30, 35], [1000, -30, 40]
        ])
        y_fuel = np.array([
            1000, 800,
            5500, 4500,
            11000, 9000
        ])
        self.fuel_model.fit(X_fuel, y_fuel)

    def predict_weather_impact(self, route_segment: str) -> dict:
        """
        Simulates predicting weather impact for a route segment.
        Returns severity (0-10) and description.
        """
        print(f"  [TOOL: Weather Predictor] Predicting weather for: {route_segment}...")
        time.sleep(0.1) # Simulate tool execution time
        severity = random.randint(0, 8)
        descriptions = [
            "Clear skies", "Light winds", "Scattered clouds", "Minor turbulence expected",
            "Moderate headwinds", "Light rain", "Potential for moderate icing",
            "Strong crosswinds", "Thunderstorm activity possible"
        ]
        description = random.choice(descriptions) if severity < 6 else "Significant weather system (e.g., thunderstorms, heavy icing)"
        return {"severity": severity, "description": description}

    def predict_fuel_consumption(self, distance_km: float, wind_component_knots: float, altitude_fl: float) -> float:
        """
        Predicts fuel consumption based on distance, wind, and altitude using a trained model.
        """
        print(f"  [TOOL: Fuel Predictor] Predicting fuel for {distance_km}km with {wind_component_knots}kt wind at FL{altitude_fl/100}...")
        time.sleep(0.1) # Simulate tool execution time
        features = np.array([[distance_km, wind_component_knots, altitude_fl/1000]])
        predicted_fuel = self.fuel_model.predict(features)[0]
        predicted_fuel *= (1 + random.uniform(-0.05, 0.05)) # Add some randomness
        return max(500, predicted_fuel) # Ensure minimum fuel

# --- 3. Agentic AI (The Orchestrator with explicit thought process) ---

class FlightPlanningAgent:
    def __init__(self):
        self.predictive_models = PredictiveModels()
        self.current_plan = {"origin": "", "destination": "", "route": [], "estimated_fuel": 0, "weather_summary": "N/A", "status": "Initialized"}
        self.strategy = "Balanced Planning" # Default strategy

    def plan_flight(self, origin: str, destination: str, user_request: str):
        self.current_plan["origin"] = origin
        self.current_plan["destination"] = destination
        self.current_plan["status"] = "Planning"
        print(f"\n--- Flight Planning Agent (Gemini 2.5) Initiated ---")
        print(f"User Request: '{user_request}'")

        # AGENT STEP 1: Use LLM to understand intent and set high-level strategy
        print("\n[AGENT THOUGHT] I need to understand the user's primary goal. Asking the LLM (AGENTIC_MODEL) for initial strategy guidance.")
        llm_strategy_prompt = (
            f"You are a flight planning assistant. A user wants to fly from {origin} to {destination}. "
            f"Their specific request is: '{user_request}'. "
            "Based on this, identify the primary planning strategy. "
            "Respond concisely with 'STRATEGY: [Strategy Name]'. "
            "Examples: 'STRATEGY: Speed Optimization', 'STRATEGY: Weather Avoidance', 'STRATEGY: Fuel Efficiency Optimization', 'STRATEGY: Balanced Planning'."
        )
        # Call without temperature
        llm_response = call_gemini_llm(AGENTIC_MODEL, llm_strategy_prompt)

        if "STRATEGY:" in llm_response:
            self.strategy = llm_response.split("STRATEGY:")[1].strip()
            print(f"[AGENT] LLM has identified strategy: {self.strategy}")
        else:
            print(f"[AGENT ERROR] LLM failed to provide a clear strategy ({llm_response}). Defaulting to Balanced Planning.")
            self.strategy = "Balanced Planning"

        # Define simulated route options
        route_options = {
            "direct_route": {"segments": ["Departure-Direct", "Mid-route-Direct", "Approach-Direct"], "distance": 4000, "altitude": 38000},
            "northern_route": {"segments": ["Departure-North", "Northern Leg", "Approach-North"], "distance": 4300, "altitude": 36000},
            "southern_route": {"segments": ["Departure-South", "Southern Leg", "Approach-South"], "distance": 4200, "altitude": 37000}
        }
        selected_route_name = "direct_route" # Default for balanced/speed

        # AGENT STEP 2: Execute strategy using predictive models (tools) and LLM for evaluation
        if "Weather Avoidance" in self.strategy:
            print(f"\n[AGENT THOUGHT] Strategy is '{self.strategy}'. I will now evaluate various routes for weather impact using my predictive models.")
            weather_impacts = {}
            for name, details in route_options.items():
                total_severity = 0
                print(f"[AGENT] Evaluating route option: {name}")
                for segment in details["segments"]:
                    # AGENT ACTION: Call Predictive Analytics Tool (Weather Predictor)
                    weather_data = self.predictive_models.predict_weather_impact(f"{origin}-{destination} ({segment})")
                    total_severity += weather_data["severity"]
                    print(f"    - Segment '{segment}': {weather_data['description']} (Severity: {weather_data['severity']})")
                weather_impacts[name] = total_severity
                print(f"  Total weather severity for {name}: {total_severity}")

            # AGENT ACTION: Ask LLM (AGENTIC_MODEL) to help evaluate and recommend the best route
            print(f"\n[AGENT THOUGHT] I have gathered weather data. Asking the LLM to help decide the best route based on weather avoidance.")
            llm_eval_prompt = (
                f"Given these routes and their total weather severity scores: {json.dumps(weather_impacts)}. "
                f"Our primary strategy is '{self.strategy}'. "
                "Which route is best, and why? Respond concisely with the recommended route name and a brief reason. "
                "Example: 'RECOMMENDATION: direct_route (least severity)'"
            )
            # Call without temperature
            llm_eval_response = call_gemini_llm(AGENTIC_MODEL, llm_eval_prompt)
            print(f"[LLM] {llm_eval_response}")

            if "RECOMMENDATION:" in llm_eval_response:
                try:
                    # Parse the recommendation. LLM's might deviate, so a robust parse is good.
                    recommended_part = llm_eval_response.split("RECOMMENDATION:")[1].strip().split(' ')[0]
                    if recommended_part in route_options:
                        selected_route_name = recommended_part
                    else:
                        print(f"[AGENT WARNING] LLM recommended an unknown route: {recommended_part}. Defaulting to least severe numerical option.")
                        sorted_routes = sorted(weather_impacts.items(), key=lambda item: item[1])
                        selected_route_name = sorted_routes[0][0]
                except Exception as e:
                    print(f"[AGENT ERROR] Failed to parse LLM recommendation: {e}. Defaulting to least severe numerical option.")
                    sorted_routes = sorted(weather_impacts.items(), key=lambda item: item[1])
                    selected_route_name = sorted_routes[0][0]
            else:
                print(f"[AGENT ERROR] LLM did not provide a clear recommendation. Defaulting to least severe numerical option.")
                sorted_routes = sorted(weather_impacts.items(), key=lambda item: item[1])
                selected_route_name = sorted_routes[0][0]

            print(f"[AGENT] Final selection for weather avoidance: {selected_route_name}")

        elif "Fuel Efficiency Optimization" in self.strategy:
            print(f"\n[AGENT THOUGHT] Strategy is '{self.strategy}'. I will evaluate routes for fuel efficiency using my predictive models.")
            fuel_costs = {}
            for name, details in route_options.items():
                simulated_wind = random.uniform(-20, 30) # Simulating dynamic wind input
                # AGENT ACTION: Call Predictive Analytics Tool (Fuel Predictor)
                predicted_fuel = self.predictive_models.predict_fuel_consumption(details["distance"], simulated_wind, details["altitude"])
                fuel_costs[name] = predicted_fuel
                print(f"  Route '{name}' predicted fuel: {predicted_fuel:.2f} liters (simulated wind: {simulated_wind}kts)")

            # AGENT ACTION: Ask LLM (AGENTIC_MODEL) to help evaluate and recommend the best route
            print(f"\n[AGENT THOUGHT] I have gathered fuel data. Asking the LLM to help decide the best route based on fuel efficiency.")
            llm_eval_prompt = (
                f"Given these routes and their total fuel costs: {json.dumps(fuel_costs)}. "
                f"Our primary strategy is '{self.strategy}'. "
                "Which route is best, and why? Respond concisely with the recommended route name and a brief reason. "
                "Example: 'RECOMMENDATION: direct_route (lowest fuel)'"
            )
            # Call without temperature
            llm_eval_response = call_gemini_llm(AGENTIC_MODEL, llm_eval_prompt)
            print(f"[LLM] {llm_eval_response}")

            if "RECOMMENDATION:" in llm_eval_response:
                try:
                    recommended_part = llm_eval_response.split("RECOMMENDATION:")[1].strip().split(' ')[0]
                    if recommended_part in route_options:
                        selected_route_name = recommended_part
                    else:
                        print(f"[AGENT WARNING] LLM recommended an unknown route: {recommended_part}. Defaulting to lowest fuel numerical option.")
                        sorted_routes = sorted(fuel_costs.items(), key=lambda item: item[1])
                        selected_route_name = sorted_routes[0][0]
                except Exception as e:
                    print(f"[AGENT ERROR] Failed to parse LLM recommendation: {e}. Defaulting to lowest fuel numerical option.")
                    sorted_routes = sorted(fuel_costs.items(), key=lambda item: item[1])
                    selected_route_name = sorted_routes[0][0]
            else:
                print(f"[AGENT ERROR] LLM did not provide a clear recommendation. Defaulting to lowest fuel numerical option.")
                sorted_routes = sorted(fuel_costs.items(), key=lambda item: item[1])
                selected_route_name = sorted_routes[0][0]

            print(f"[AGENT] Final selection for fuel efficiency: {selected_route_name}")

        elif "Speed Optimization" in self.strategy:
            print(f"\n[AGENT THOUGHT] Strategy is '{self.strategy}'. For speed, I'll generally favor the most direct route.")
            selected_route_name = "direct_route" # Agent's simple heuristic for speed
            print(f"[AGENT] Selecting direct route for speed optimization: {selected_route_name}")

        else: # Balanced Planning
            print(f"\n[AGENT THOUGHT] Strategy is '{self.strategy}'. Balancing factors, defaulting to the direct route and will provide a balanced summary.")
            selected_route_name = "direct_route"


        # AGENT STEP 3: Finalize plan details with selected route
        selected_route = route_options[selected_route_name]
        self.current_plan["route"] = selected_route["segments"]

        # AGENT ACTION: Call Fuel Predictor for the chosen route
        final_wind = random.uniform(-15, 25) # Another simulated wind for final calc
        self.current_plan["estimated_fuel"] = self.predictive_models.predict_fuel_consumption(
            selected_route["distance"], final_wind, selected_route["altitude"]
        )

        # AGENT ACTION: Call Weather Predictor for chosen route for final summary
        final_weather_summary_list = []
        for segment in selected_route["segments"]:
            weather_data = self.predictive_models.predict_weather_impact(f"{origin}-{destination} ({segment})")
            final_weather_summary_list.append(f"{segment}: {weather_data['description']}")
        self.current_plan["weather_summary"] = "; ".join(final_weather_summary_list)


        # AGENT STEP 4: Use LLM (RESPONDER_MODEL) to generate the final human-readable flight plan report
        print(f"\n[AGENT THOUGHT] All data gathered. Now, I will use the LLM (RESPONDER_MODEL) to synthesize this into a comprehensive, human-readable flight plan report.")
        final_report_prompt = (
            f"Generate a concise, professional flight plan summary based on the following details. "
            f"Emphasize the chosen strategy and its implications for the plan.\n\n"
            f"Origin: {origin}\n"
            f"Destination: {destination}\n"
            f"Planning Strategy: {self.strategy}\n"
            f"Selected Route: {', '.join(self.current_plan['route'])}\n"
            f"Estimated Fuel Required: {self.current_plan['estimated_fuel']:.2f} liters\n"
            f"Key Weather Summary: {self.current_plan['weather_summary']}\n\n"
            "Ensure the summary is clear, concise, and highlights how the strategy shaped the route and resources."
        )
        # Using RESPONDER_MODEL for final output to show distinct roles if desired, though AGENTIC_MODEL could also do this.
        # Removed temperature from this call as well
        final_plan_text = call_gemini_llm(RESPONDER_MODEL, final_report_prompt)
        self.current_plan["detailed_plan"] = final_plan_text
        self.current_plan["status"] = "Planned"

        print("\n--- Final Flight Plan (Constructed by Agent, Presented by Gemini 2.5 LLM) ---")
        print(self.current_plan['detailed_plan'])
        print("\n------------------------------")


# --- Demo Execution ---
if __name__ == "__main__":
    agent = FlightPlanningAgent()

    print("Demo 1: Standard Flight Plan")
    agent.plan_flight("Montreal", "Vancouver", "standard flight")

    print("\n" + "="*70 + "\n")

    print("Demo 2: Flight Plan - Avoid Adverse Weather")
    agent.plan_flight("Montreal", "Vancouver", "avoid adverse weather as much as possible, safety first")

    print("\n" + "="*70 + "\n")

    print("Demo 3: Flight Plan - Optimize for Fuel Efficiency")
    agent.plan_flight("Montreal", "Vancouver", "find the most fuel-efficient route, even if it adds a bit of time")

    print("\n" + "="*70 + "\n")

    print("Demo 4: Flight Plan - Optimize for Speed")
    agent.plan_flight("Montreal", "Vancouver", "get there as fast as possible, speed is paramount")

Google Generative AI configured successfully using Colab Secrets.
Gemini model 'gemini-2.5-flash' initialized for agentic and response generation.
Demo 1: Standard Flight Plan

--- Flight Planning Agent (Gemini 2.5) Initiated ---
User Request: 'standard flight'

[AGENT THOUGHT] I need to understand the user's primary goal. Asking the LLM (AGENTIC_MODEL) for initial strategy guidance.

[LLM] Calling Gemini model with prompt (first 100 chars): "You are a flight planning assistant. A user wants to fly from Montreal to Vancouver. Their specific ..."
[AGENT] LLM has identified strategy: Balanced Planning

[AGENT THOUGHT] Strategy is 'Balanced Planning'. Balancing factors, defaulting to the direct route and will provide a balanced summary.
  [TOOL: Fuel Predictor] Predicting fuel for 4000km with -6.555861607993769kt wind at FL380.0...
  [TOOL: Weather Predictor] Predicting weather for: Montreal-Vancouver (Departure-Direct)...
  [TOOL: Weather Predictor] Predicting weather for: Montreal-Vanco